# MHyEEG: DEAP DATASET

### 0: IMPORTING DEAP DATASET AND UNZIPPING

In [6]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.3 MB/s eta 0:00:00


In [1]:
from google.colab import drive

# Mount the Google Drive
drive.mount('/content/drive')

# Verify the drive is mounted
!ls /content/drive/My\ Drive


Mounted at /content/drive
'ACSAI_Marks&Recommendations.pdf'  'MasterThesis_Proposal_(A.Borgi).gslides'
 Alessio_Borgi_CV.pdf		    nano-imagenet-30-Original
 Alessio_Borgi_CV_Short.pdf	    NN
'Colab Notebooks'		    open_window.py
 data				    Open_Window.traj
 DEAP.zip			    problem.txt
 HW2				    sapienza-ppt-template_dark.gslides
 identita.pdf			    StyleAligned
 Iscrizione.pdf			    StyleAligned_M2L.gslides
 KAN-GAT_EAI.gslides		   'Tony Starks Concentration Mix.mp3'
 Linz				    XGNNs_RL.gslides


In [2]:
import zipfile
import os

def unzip_file(zip_path, extract_to):
    """
    Unzips a file to a specified directory.

    Args:
    - zip_path (str): The path to the zip file.
    - extract_to (str): The directory where files should be extracted.
    """
    # Ensure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)

    # Open and extract the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Extracted all files to: {extract_to}")

# Example usage
zip_path = "/content/drive/MyDrive/DEAP.zip"  # Replace with your zip file path
extract_to = "/content/MHyEEG/data/"  # Replace with your target extraction folder
unzip_file(zip_path, extract_to)


Extracted all files to: /content/MHyEEG/data/


In [3]:
import os
import zipfile

def unzip_all_in_directory(directory):
    """
    Unzips all zip files in the given directory and its subdirectories.

    Args:
    - directory (str): The directory to search for zip files.
    """
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                extract_to = os.path.join(root, file.replace('.zip', ''))
                os.makedirs(extract_to, exist_ok=True)
                print(f"Unzipping: {zip_path} to {extract_to}")
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)
                print(f"Extracted: {zip_path} to {extract_to}")

# Directory containing the DEAP dataset
deap_data_dir = "/content/MHyEEG/data/DEAP"
unzip_all_in_directory(deap_data_dir)


Unzipping: /content/MHyEEG/data/DEAP/metadata_csv.zip to /content/MHyEEG/data/DEAP/metadata_csv
Extracted: /content/MHyEEG/data/DEAP/metadata_csv.zip to /content/MHyEEG/data/DEAP/metadata_csv
Unzipping: /content/MHyEEG/data/DEAP/face_video.zip to /content/MHyEEG/data/DEAP/face_video
Extracted: /content/MHyEEG/data/DEAP/face_video.zip to /content/MHyEEG/data/DEAP/face_video
Unzipping: /content/MHyEEG/data/DEAP/data_original.zip to /content/MHyEEG/data/DEAP/data_original
Extracted: /content/MHyEEG/data/DEAP/data_original.zip to /content/MHyEEG/data/DEAP/data_original
Unzipping: /content/MHyEEG/data/DEAP/data_preprocessed_python.zip to /content/MHyEEG/data/DEAP/data_preprocessed_python
Extracted: /content/MHyEEG/data/DEAP/data_preprocessed_python.zip to /content/MHyEEG/data/DEAP/data_preprocessed_python




### 1: PRE-PROCESSING DEAP EEG DATA

This section describes the preprocessing pipeline for the DEAP EEG dataset. Each step is explained in detail, including the purpose and reasoning behind the operations, updated to handle mismatched channels and improve resilience.

---

##### **1. Set Channel Types for Non-EEG Channels**
The DEAP dataset contains non-EEG channels such as:
- **External electrodes (EXG1–EXG8):** Measure eye movements.
- **Galvanic Skin Response (GSR1, GSR2):** Tracks skin conductance.
- **Respiratory (Resp):** Monitors breathing patterns.
- **Temperature (Temp):** Records skin temperature.

To avoid interference in EEG-specific steps, these channels are assigned the type `'misc'` (miscellaneous). This allows them to remain in the dataset without being used in operations specific to EEG data.

---

##### **2. Set Montage for EEG Channels**
A montage defines the electrode positions on the scalp, mapping signals to specific physical locations. The DEAP dataset uses the Biosemi 32-channel system. The montage ensures that:
- EEG channels are properly represented in the analysis.
- Non-EEG channels (e.g., GSR, Respiration) are ignored during this step.

###### **Handling Mismatched Channels:**
- If channels in the raw data are missing or do not match the Biosemi32 montage, they are logged, and the processing continues.
- The parameter `on_missing='ignore'` ensures that unmatched channels do not interrupt the pipeline.
- This approach allows the processing of data for all subjects, even if specific channels are missing or mislabeled.

---

##### **3. Apply Bandpass Filter**
A bandpass filter is used to retain only the relevant EEG frequencies (1–45 Hz):
- **Low cutoff (1 Hz):** Removes slow drifts and baseline noise.
- **High cutoff (45 Hz):** Removes high-frequency artifacts such as muscle activity.

The filtered signal \( X(t) \) can be represented as:
$$
X_{\text{filtered}}(t) = \int_{1\,\text{Hz}}^{45\,\text{Hz}} X(f) e^{j2\pi ft} \, df
$$
where \( X(f) \) is the Fourier transform of the raw signal.

---

##### **4. Resample the Data**
The original sampling rate of the DEAP dataset is 512 Hz. Resampling reduces the rate to 128 Hz:
- **Purpose:** Lower computational requirements while preserving the signal's temporal and spectral integrity.

---

##### **5. Apply ICA for Artifact Removal**
Independent Component Analysis (ICA) decomposes the EEG signal into independent components, separating brain activity from artifacts (e.g., eye blinks, muscle movements). The process involves:
1. Estimating independent sources \( S \) from the observed signals \( X \):
   $$
   X = A S
   $$
   where \( A \) is the mixing matrix.
2. Applying the inverse of \( A \) to remove unwanted components:
   $$
   S_{\text{cleaned}} = A^{-1} X
   $$

Artifacts are identified and removed based on their statistical properties (e.g., amplitude and frequency patterns).

---

##### **Summary of Workflow**
1. **Handle Non-EEG Channels:** Assign non-EEG channels to the `'misc'` type to prevent errors.
2. **Apply Montage:** Map EEG signals to their standard scalp positions using the Biosemi 32-channel system.
   - **Updated Handling for Mismatched Channels:** If channels are missing or unmatched, they are ignored using `on_missing='ignore'`.
3. **Filter Data:** Isolate the EEG frequency range (1–45 Hz) to improve signal quality.
4. **Resample Data:** Downsample to 128 Hz to reduce computational complexity.
5. **Remove Artifacts:** Use ICA to clean the EEG signals from noise such as eye blinks and muscle artifacts.

This updated preprocessing pipeline ensures compatibility with the DEAP dataset, handling potential mismatches and improving robustness for high-quality analysis.


In [9]:
import os
import mne
from mne.preprocessing import ICA

def load_raw_data(data_dir, subject_id):
    """
    Load raw DEAP EEG data from the unzipped directory structure.
    Args:
        data_dir (str): Path to the `data_original` directory.
        subject_id (int): Subject ID (1-32).
    Returns:
        raw (mne.io.Raw): Raw EEG data.
    """
    file_path = os.path.join(data_dir, f"s{str(subject_id).zfill(2)}.bdf")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    print(f"Loading data for Subject {subject_id} from {file_path}")
    raw = mne.io.read_raw_bdf(file_path, preload=True)
    return raw

def preprocess_raw_data(raw):
    """
    Preprocess raw EEG data: set montage, filter, resample, and apply ICA.
    Args:
        raw (mne.io.Raw): Raw EEG data.
    Returns:
        preprocessed_raw (mne.io.Raw): Preprocessed EEG data.
    """
    # Set channel types for non-EEG channels
    non_eeg_channels = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8',
                        'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']
    raw.set_channel_types({ch: 'misc' for ch in non_eeg_channels})

    # Match channel names to Biosemi32 montage (if possible)
    montage = mne.channels.make_standard_montage("biosemi32")
    raw_channels = raw.info['ch_names']
    montage_channels = montage.ch_names

    # Identify mismatched channels
    missing_channels = [ch for ch in raw_channels if ch not in montage_channels]
    print(f"Missing or unmatched channels: {missing_channels}")

    # Attempt to set montage, allowing for missing channels
    raw.pick_types(eeg=True)  # Select only EEG channels for montage
    try:
        raw.set_montage(montage, on_missing='ignore')
    except ValueError as e:
        print(f"Error setting montage: {e}. Proceeding without strict mapping.")

    # Filter data (band-pass 1-45 Hz)
    raw.filter(l_freq=1.0, h_freq=45.0)

    # Downsample to 128 Hz
    raw.resample(128)

    # Apply ICA for artifact removal
    ica = ICA(n_components=15, random_state=97, max_iter=800)
    ica.fit(raw)
    raw = ica.apply(raw)

    return raw

def save_preprocessed_data(raw, output_dir, subject_id):
    """
    Save preprocessed EEG data to FIF file format.
    Args:
        raw (mne.io.Raw): Preprocessed EEG data.
        output_dir (str): Directory to save the preprocessed data.
        subject_id (int): Subject ID (1-32).
    """
    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, f"preprocessed_s{str(subject_id).zfill(2)}.raw.fif")
    raw.save(file_path, overwrite=True)
    print(f"Saved preprocessed data for Subject {subject_id} to {file_path}")

if __name__ == "__main__":
    # Define paths
    data_dir = "/content/MHyEEG/data/DEAP/data_original"
    output_dir = "/content/MHyEEG/data/DEAP/preprocessed"

    # Process each subject
    for subject_id in range(1, 33):
        print(f"Processing Subject {subject_id}...")
        try:
            raw = load_raw_data(data_dir, subject_id)
            preprocessed_raw = preprocess_raw_data(raw)
            save_preprocessed_data(preprocessed_raw, output_dir, subject_id)
        except Exception as e:
            print(f"An error occurred for Subject {subject_id}: {e}")


Processing Subject 1...
Loading data for Subject 1 from /content/MHyEEG/data/DEAP/data_original/s01.bdf
Extracting EDF parameters from /content/MHyEEG/data/DEAP/data_original/s01.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1980927  =      0.000 ...  3868.998 secs...
Missing or unmatched channels: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'Status']
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


<ipython-input-9-d8f647c8f685>:32: RuntimeWarning: The unit for channel(s) EXG1, EXG2, EXG3, EXG4, EXG5, EXG6, EXG7, EXG8, Erg1, Erg2, GSR1, GSR2, Plet, Resp, Temp has changed from V to NA.
  raw.set_channel_types({ch: 'misc' for ch in non_eeg_channels})


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (3.303 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.0s


Fitting ICA to data using 32 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 24.2s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 32 PCA components
Writing /content/MHyEEG/data/DEAP/preprocessed/preprocessed_s01.raw.fif
Closing /content/MHyEEG/data/DEAP/preprocessed/preprocessed_s01.raw.fif
[done]
Saved preprocessed data for Subject 1 to /content/MHyEEG/data/DEAP/preprocessed/preprocessed_s01.raw.fif
Processing Subject 2...
Loading data for Subject 2 from /content/MHyEEG/data/DEAP/data_original/s02.bdf
Extracting EDF parameters from /content/MHyEEG/data/DEAP/data_original/s02.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1895935  =      0.000 ...  3702.998 secs...
Missing or unmatched channels: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2

...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (3.303 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.3s


Fitting ICA to data using 34 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 17.4s.
Applying ICA to Raw instance


<ipython-input-9-d8f647c8f685>:58: RuntimeWarning: Using n_components=15 (resulting in n_components_=15) may lead to an unstable mixing matrix estimation because the ratio between the largest (34) and smallest (3.1e-15) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 1
  ica.fit(raw)


    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 34 PCA components
Writing /content/MHyEEG/data/DEAP/preprocessed/preprocessed_s32.raw.fif
Closing /content/MHyEEG/data/DEAP/preprocessed/preprocessed_s32.raw.fif
[done]
Saved preprocessed data for Subject 32 to /content/MHyEEG/data/DEAP/preprocessed/preprocessed_s32.raw.fif


In [ ]:
import os
import mne
from mne.preprocessing import ICA

def load_raw_data(data_dir, subject_id):
    """
    Load raw DEAP EEG data from the unzipped directory structure.
    Args:
        data_dir (str): Path to the `data_original` directory.
        subject_id (int): Subject ID (1-32).
    Returns:
        raw (mne.io.Raw): Raw EEG data.
    """
    file_path = os.path.join(data_dir, f"s{str(subject_id).zfill(2)}.bdf")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    print(f"Loading data for Subject {subject_id} from {file_path}")
    raw = mne.io.read_raw_bdf(file_path, preload=True)
    return raw

def preprocess_raw_data(raw):
    """
    Preprocess raw EEG data: set montage, filter, resample, and apply ICA.
    Args:
        raw (mne.io.Raw): Raw EEG data.
    Returns:
        preprocessed_raw (mne.io.Raw): Preprocessed EEG data.
    """
    # Handle duplicate or empty channel names
    raw.rename_channels({name: f"ch_{i}" for i, name in enumerate(raw.info["ch_names"]) if name == ''})

    # Set channel types for non-EEG channels
    non_eeg_channels = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8',
                        'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'Status']
    raw.set_channel_types({ch: 'misc' for ch in non_eeg_channels if ch in raw.info["ch_names"]})

    # Match channel names to Biosemi32 montage (if possible)
    montage = mne.channels.make_standard_montage("biosemi32")
    raw_channels = raw.info['ch_names']
    montage_channels = montage.ch_names

    # Identify mismatched channels
    missing_channels = [ch for ch in raw_channels if ch not in montage_channels]
    print(f"Missing or unmatched channels: {missing_channels}")

    # Attempt to set montage, allowing for missing channels
    raw.pick_types(eeg=True, exclude='misc')  # Select only EEG channels for montage
    try:
        raw.set_montage(montage, on_missing='ignore')
    except ValueError as e:
        print(f"Error setting montage: {e}. Proceeding without strict mapping.")

    # Filter data (band-pass 1-45 Hz)
    raw.filter(l_freq=1.0, h_freq=45.0)

    # Downsample to 128 Hz
    raw.resample(128)

    # Apply ICA for artifact removal with safer n_components
    ica = ICA(n_components=0.999999, random_state=97, max_iter=800)
    ica.fit(raw)
    raw = ica.apply(raw)

    return raw

def save_preprocessed_data(raw, output_dir, subject_id):
    """
    Save preprocessed EEG data to FIF file format.
    Args:
        raw (mne.io.Raw): Preprocessed EEG data.
        output_dir (str): Directory to save the preprocessed data.
        subject_id (int): Subject ID (1-32).
    """
    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, f"preprocessed_s{str(subject_id).zfill(2)}.raw.fif")
    raw.save(file_path, overwrite=True)
    print(f"Saved preprocessed data for Subject {subject_id} to {file_path}")

if __name__ == "__main__":
    # Define paths
    data_dir = "/content/MHyEEG/data/DEAP/data_original"
    output_dir = "/content/MHyEEG/data/DEAP/preprocessed"

    # Process each subject
    for subject_id in range(1, 33):
        print(f"Processing Subject {subject_id}...")
        try:
            raw = load_raw_data(data_dir, subject_id)
            preprocessed_raw = preprocess_raw_data(raw)
            save_preprocessed_data(preprocessed_raw, output_dir, subject_id)
        except Exception as e:
            print(f"An error occurred for Subject {subject_id}: {e}")


Processing Subject 1...
Loading data for Subject 1 from /content/MHyEEG/data/DEAP/data_original/s01.bdf
Extracting EDF parameters from /content/MHyEEG/data/DEAP/data_original/s01.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1980927  =      0.000 ...  3868.998 secs...
Missing or unmatched channels: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'Status']
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
An error occurred for Subject 1: exclude must either be "bads" or a list of strings. If only one channel is to be excluded, use [ch_name] instead of passing ch_name.
Processing Subject 2...
Loading data for Subject 2 from /content/MHyEEG/data/DEAP/data_original/s02.bdf
Extracting EDF parameters from /content/MHyEEG/data/DEAP/data_original/s02.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...

<ipython-input-10-c731d0744a29>:35: RuntimeWarning: The unit for channel(s) EXG1, EXG2, EXG3, EXG4, EXG5, EXG6, EXG7, EXG8, Erg1, Erg2, GSR1, GSR2, Plet, Resp, Temp has changed from V to NA.
  raw.set_channel_types({ch: 'misc' for ch in non_eeg_channels if ch in raw.info["ch_names"]})


Missing or unmatched channels: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'Status']
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
An error occurred for Subject 2: exclude must either be "bads" or a list of strings. If only one channel is to be excluded, use [ch_name] instead of passing ch_name.
Processing Subject 3...
Loading data for Subject 3 from /content/MHyEEG/data/DEAP/data_original/s03.bdf
Extracting EDF parameters from /content/MHyEEG/data/DEAP/data_original/s03.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1989119  =      0.000 ...  3884.998 secs...


<ipython-input-10-c731d0744a29>:35: RuntimeWarning: The unit for channel(s) EXG1, EXG2, EXG3, EXG4, EXG5, EXG6, EXG7, EXG8, Erg1, Erg2, GSR1, GSR2, Plet, Resp, Temp has changed from V to NA.
  raw.set_channel_types({ch: 'misc' for ch in non_eeg_channels if ch in raw.info["ch_names"]})
